## Ejercicio 44
Si decimos que la ubicación de un usuario es el promedio de la latitud y longitud de
los contenidos geolocalizados para los cuales editó la última versión (ignorar
usuarios que no editaron contenido geolocalizado). ¿Cuáles son los dos usuarios
más cercanos? (⭐⭐⭐)

In [1]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

In [2]:
geo_tags =  pd.read_csv('geo_tags.csv')

In [8]:
contents = pd.read_csv('contents.csv')

In [4]:
geo_tags.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 556393 entries, 0 to 556392
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   gt_id       556393 non-null  int64  
 1   gt_page_id  556393 non-null  int64  
 2   gt_globe    556393 non-null  object 
 3   gt_primary  556393 non-null  int64  
 4   gt_lat      556393 non-null  float64
 5   gt_lon      556393 non-null  float64
 6   gt_dim      556393 non-null  int64  
 7   gt_type     449893 non-null  object 
 8   gt_name     72557 non-null   object 
 9   gt_country  79192 non-null   object 
 10  gt_region   3088 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 46.7+ MB


In [9]:
contents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4132164 entries, 0 to 4132163
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   title               object 
 1   id                  int64  
 2   namespace           int64  
 3   revision_id         int64  
 4   parent_revision     float64
 5   revision_timestamp  object 
 6   revisor_username    object 
 7   revisor_id          float64
 8   revisor_ip          object 
 9   revisor_comment     object 
dtypes: float64(2), int64(3), object(5)
memory usage: 315.3+ MB


In [10]:
contents.head()

,title,id,namespace,revision_id,parent_revision,revision_timestamp,revisor_username,revisor_id,revisor_ip,revisor_comment
0,Wikipedia:Artículos solicitados,5,4,132533307,132516631.0,2021-01-19T18:40:34Z,Metrónomo,1708233.0,NaN,Así esta mejor
1,Andorra,7,0,138035057,138027475.0,2021-08-31T18:27:01Z,NaN,NaN,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2,Añadir contenido
2,Argentina,10,0,137966826,137825165.0,2021-08-28T17:17:54Z,Traitrich,4642283.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
3,Geografía de Andorra,15,0,137986788,133666383.0,2021-08-29T17:16:00Z,Lojwe,5338508.0,NaN,Reemplazos con [[Usuario:Benjavalero/Replacer|...
4,Demografía de Andorra,17,0,130869554,129413188.0,2020-11-12T19:56:15Z,Jialxv,3615366.0,NaN,Correcciones ortográficas con [[Usuario:Benjav...


In [11]:
geo_tags.head()

,gt_id,gt_page_id,gt_globe,gt_primary,gt_lat,gt_lon,gt_dim,gt_type,gt_name,gt_country,gt_region
0,158041,4328020,earth,0,41.5075,-5.8175,1000,NaN,NaN,NaN,NaN
1,5844377,4670424,earth,0,13.7380,-89.2920,10000,city,NaN,NaN,NaN
2,5844381,4670424,earth,0,13.7020,-89.2080,10000,city,NaN,NaN,NaN
3,5844383,4670424,earth,0,13.4000,-89.7500,10000,city,NaN,NaN,NaN
4,5844388,4670424,earth,0,12.8500,-88.0000,10000,city,NaN,NaN,NaN


In [28]:
contents_short = contents[['id','revision_id','revision_timestamp']]
contents_short.head()

,id,revision_id,revision_timestamp
0,5,132533307,2021-01-19T18:40:34Z
1,7,138035057,2021-08-31T18:27:01Z
2,10,137966826,2021-08-28T17:17:54Z
3,15,137986788,2021-08-29T17:16:00Z
4,17,130869554,2020-11-12T19:56:15Z


In [29]:
contents_short['revisor_id/ip'] = contents['revisor_id'].fillna(contents['revisor_ip'])
contents_short.head()

<ipython-input-29-470df0935a65>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents_short['revisor_id/ip'] = contents['revisor_id'].fillna(contents['revisor_ip'])


,id,revision_id,revision_timestamp,revisor_id/ip
0,5,132533307,2021-01-19T18:40:34Z,1.70823e+06
1,7,138035057,2021-08-31T18:27:01Z,2800:CD0:FC1B:A900:C5A4:9E81:7175:2ED2
2,10,137966826,2021-08-28T17:17:54Z,4.64228e+06
3,15,137986788,2021-08-29T17:16:00Z,5.33851e+06
4,17,130869554,2020-11-12T19:56:15Z,3.61537e+06


In [30]:
contents_short['revision_timestamp'] = pd.to_datetime(contents_short['revision_timestamp'])
contents_short.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4132164 entries, 0 to 4132163
Data columns (total 4 columns):
 #   Column              Dtype              
---  ------              -----              
 0   id                  int64              
 1   revision_id         int64              
 2   revision_timestamp  datetime64[ns, UTC]
 3   revisor_id/ip       object             
dtypes: datetime64[ns, UTC](1), int64(2), object(1)
memory usage: 126.1+ MB


<ipython-input-30-c5fe3af0d051>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents_short['revision_timestamp'] = pd.to_datetime(contents_short['revision_timestamp'])


In [31]:
contents_short.isnull().sum()

id                    0
revision_id           0
revision_timestamp    0
revisor_id/ip         1
dtype: int64

In [32]:
# Como tengo un solo null, lo dropeo
contents_short.dropna(inplace=True)

<ipython-input-32-f1728d3030d1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  contents_short.dropna(inplace=True)


In [33]:
contents_short.isnull().sum()

id                    0
revision_id           0
revision_timestamp    0
revisor_id/ip         0
dtype: int64

In [18]:
geo_tags_short =  geo_tags[['gt_page_id','gt_lat','gt_lon']]
geo_tags_short.isnull().sum()

gt_page_id    0
gt_lat        0
gt_lon        0
dtype: int64

Como podemos ver, ambos dt con los que voy a trabajar estan limpios.

In [19]:
# Cambio el nombre de la columna
geo_tags_short.rename(columns={'gt_page_id':'id'},inplace=True)

C:\Users\alejo\anaconda3\lib\site-packages\pandas\core\frame.py:4125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [34]:
# Mergeo los geo_tags con mis contenidos para quedarme con aquellos que poseen una referencia geolocalizable
merged_content = geo_tags_short.merge(contents_short,on='id')
merged_content.head()

,id,gt_lat,gt_lon,revision_id,revision_timestamp,revisor_id/ip
0,4328020,41.507500,-5.817500,65728616,2013-03-29 10:02:04+00:00,622035
1,4328020,41.751667,-5.751667,65728616,2013-03-29 10:02:04+00:00,622035
2,4328020,41.492500,-5.786111,65728616,2013-03-29 10:02:04+00:00,622035
3,4328020,41.492222,-5.771111,65728616,2013-03-29 10:02:04+00:00,622035
4,4328020,41.470833,-5.757778,65728616,2013-03-29 10:02:04+00:00,622035


In [36]:
# Ahora tengo en un DataFrame a los usuaarios que hicieron la ultima revision, desde donde y para que contenido.
# Por lo que voy a agruparlos en un groupby para buscar los dos mas cercanos.
grouped_revisor = merged_content.groupby(['revisor_id/ip']).agg({'gt_lat':'mean','gt_lon':'mean'})
grouped_revisor

,gt_lat,gt_lon
revisor_id/ip,,
639.0,32.747667,-62.459055
1299.0,40.223412,-4.529800
1430.0,41.538595,-1.630543
1640.0,27.508962,-16.366295
1854.0,38.412029,-0.988330
...,...,...
98.158.240.204,20.651622,-103.400247
99.164.33.227,-11.844682,-76.383777
99.2.37.220,38.622500,-90.193056


In [38]:
# Para poder usar neigthboors
coordenates = grouped_revisor[['gt_lat','gt_lon']].values
coordenates

array([[ 32.74766667, -62.45905511],
       [ 40.22341195,  -4.52979967],
       [ 41.538595  ,  -1.630543  ],
       ...,
       [ 38.6225    , -90.193056  ],
       [ 47.49388889,   7.31583333],
       [ 27.19      , -82.37      ]])

In [39]:
naam = NearestNeighbors(n_neighbors=2, algorithm='kd_tree').fit(coordenates)
distances, indices = naam.kneighbors(coordenates)

In [42]:
distances

array([[0.        , 0.17815216],
       [0.        , 0.07995968],
       [0.        , 0.00696172],
       ...,
       [0.        , 0.00667792],
       [0.        , 0.10592115],
       [0.        , 0.8001235 ]])

In [43]:
# Posiciones de indices donde se encuentran posibles vecinos cercanos
i = np.where(distances[:,1].flatten() ==  np.amin(distances[:,1].flatten()))
i

(array([  288,   674,  1129,  1509,  1637,  1835,  1845,  1875,  2055,
         2186,  2227,  2228,  2753,  2783,  2848,  2893,  2905,  3245,
         3288,  3447,  3574,  3579,  3656,  3679,  3698,  3704,  3782,
         3821,  3946,  4093,  4334,  4416,  4516,  4557,  5053,  5097,
         5156,  5276,  5310,  5457,  5511,  5523,  5526,  5560,  5580,
         5749,  5813,  5817,  5870,  6028,  6095,  6178,  6273,  6314,
         6422,  6606,  6638,  6807,  6808,  6881,  6923,  6952,  7041,
         7117,  7307,  7467,  7544,  7586,  7651,  7670,  7699,  7725,
         7741,  7749,  7922,  8021,  8138,  8159,  8362,  8500,  8563,
         8799,  8859,  9053,  9190,  9224,  9276,  9299,  9380,  9404,
         9537,  9563,  9589,  9679,  9791,  9837,  9977, 10037, 10195,
        10255, 10306, 10366, 10368, 10379, 10380, 10407, 10436, 10553,
        10580, 10590, 10637, 10677, 10850, 10986, 11036, 11145, 11282,
        11323, 11351, 11503, 11861, 11960, 12065, 12066, 12188, 12288,
      

In [44]:
# Elijo los de la posicion 0, asegurandome que son los dos mas cercanos entre si.
grouped_revisor.iloc[indices[0]]

,gt_lat,gt_lon
revisor_id/ip,,
639.0,32.747667,-62.459055
458083.0,32.890752,-62.565191
